## Обучение моделей (3)

В этом блокноте используются файлы, полученные при помощи предыдущих блокнотов (тренировочный и тестовый наборы данных). Наборы векторизуются, к полученным разреженным матрицам добавляются новые признаки, а затем на полученных данных обучается логистическая регресиия и применяется для классификации тестовых сессий.

**Содержание**
1. Загрузка предподготовленных данных
2. Применение CountVectorizer и TfidfVectorizer
3. Функции для добавления фичей
4. Применение преобразованных наборов данных для классификации
5. Применение новых фичей для классификации
6. Заключение

### Загрузка предподготовленных данных

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm_notebook
import pickle

import numpy as np
import pandas as pd

import scipy.sparse as sp

from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import (SGDClassifier, LogisticRegression, 
                                  LogisticRegressionCV)
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

PATH_TO_DATASET = os.path.join('intermediate_data', 'test_train')
PATH_TO_DICT = os.path.join('initial_data', 'site_dict')

def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    ''' Записывает файл для посылки в Kaggle '''

    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [2]:
with open(os.path.join(PATH_TO_DATASET, 'train_s10_w10_m30_final.pkl'), 'rb') as f:
    train_df = pickle.load(f)

with open(os.path.join(PATH_TO_DATASET, 'test_s10_w10_m30_final.pkl'), 'rb') as f:
    test_df = pickle.load(f)

train_df = train_df.sort_values(by=['year', 'month', 'day', 'time'])

In [3]:
train_test_df = pd.concat([train_df, test_df])
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype('int')
y_train = train_df['target'].astype('int').values

### Применение CountVectorizer и TfidfVectorizer

In [48]:
sites = ['site%s' % i for i in range(1, 11)]

train_df[sites].fillna(0).astype('int').to_csv(os.path.join('intermediate_data', 'train_sessions_text_cnt.txt'), 
                                               sep=' ', 
                       index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv(os.path.join('intermediate_data', 'test_sessions_text_cnt.txt'), 
                                              sep=' ', 
                       index=None, header=None)

with open(os.path.join(PATH_TO_DICT, 'site_dict.pkl'), 'rb') as file:
    sites_dict = pickle.load(file)
reversed_dict = {v: k for k, v in sites_dict.items()}
reversed_dict[0] = '0'

train_df[sites].fillna(0).astype('int').applymap(
    lambda x: reversed_dict[x]).to_csv(os.path.join('intermediate_data', 'train_sessions_text_tfidf.txt'), sep=' ', 
    index=None, header=None)
test_df[sites].fillna(0).astype('int').applymap(
    lambda x: reversed_dict[x]).to_csv(os.path.join('intermediate_data', 'test_sessions_text_tfidf.txt'), sep=' ', 
    index=None, header=None)

In [49]:
%%time
cnt_vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=50000)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 5), max_features=100000, tokenizer=lambda s: s.split())
with open(os.path.join('intermediate_data', 'train_sessions_text_cnt.txt')) as inp_train_file:
    X_train = cnt_vectorizer.fit_transform(inp_train_file)
with open(os.path.join('intermediate_data', 'train_sessions_text_tfidf.txt')) as inp_train_file:
    X_train_tfidf = tfidf_vectorizer.fit_transform(inp_train_file)
with open(os.path.join('intermediate_data', 'test_sessions_text_cnt.txt')) as inp_test_file:
    X_test = cnt_vectorizer.transform(inp_test_file)
with open(os.path.join('intermediate_data', 'test_sessions_text_tfidf.txt')) as inp_test_file:
    X_test_tfidf = tfidf_vectorizer.transform(inp_test_file)

Wall time: 35.6 s


In [50]:
cnt_vectorizer.get_feature_names()[:10]

['10',
 '10 1086',
 '10 11',
 '10 11 11',
 '10 11 12',
 '10 11 14',
 '10 11 15',
 '10 11241',
 '10 1199',
 '10 12']

In [51]:
tfidf_vectorizer.get_feature_names()[:10]

['0',
 '0 0',
 '0 0 0',
 '0 0 0 0',
 '0 0 0 0 0',
 '0.academia-assets.com',
 '0.docs.google.com',
 '0.docs.google.com 0',
 '0.docs.google.com 0 0',
 '0.docs.google.com 0 0 0']

### Функции для добавления фичей

In [8]:
def add_time_features(df, X_sparse):
    ''' Добавляет индикаторы (4 признака) утра, дня, вечера и ночи '''

    hour = df['start_hour']
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    X = sp.hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [9]:
def add_session_timespan(df, X_sparse):
    ''' Добавляет продолжительность сессии в секундах (1 признак) '''

    session_timespan = df['session_timespan']
    X = sp.hstack([X_sparse, session_timespan.values.reshape(-1, 1)])
    return X
    

In [10]:
def feature_short_session(df, X_sparse, time_min=0, time_max=3):
    ''' Добавляет индикаторы короткой сессии (от 0 до 2 секунд) 
    (1 признак)'''

    def short_session(x): 
        if ((x >= time_min) and (x < time_max)):
            return 1
        return 0

    times = df['session_timespan']
    X = sp.hstack([X_sparse, times.apply(short_session).values.reshape(-1, 1)])
    return X
    

def feature_middle_session(df, X_sparse, time_min=3, time_max=9):
    ''' Добавляет индикаторы средней сессии (от 3 до 6 секунд) 
    (1 признак)'''

    def middle_session(x): 
        if ((x >= time_min) and (x < time_max)):
            return 1
        return 0

    times = df['session_timespan']
    X = sp.hstack([X_sparse, times.apply(middle_session).values.reshape(-1, 1)])
    return X

In [11]:
def add_top_sites(df, X_sparse, list_of_sites={3000, 2080, 27307}):
    ''' Добавляет индикаторы (1 признак) нескольких сайтов, которые Alice 
    использует замето чаще, чем остальные люди '''
    
    sites = df[['site%s' % i for i in range(1, 11)]].values
    result = [0]*sites.shape[0]

    for i, session in enumerate(sites):
        for site in session:
            if site in list_of_sites:
                result[i] += 1
    
    X = sp.hstack([X_sparse, np.array(result).reshape(-1, 1)])
    return X

In [12]:
def add_reverse_top_sites(df, X_sparse, list_of_sites={55, 56, 570, 
                                                       778, 780, 782, 
                                                       786, 812}):
    ''' Добавляет индикаторы (1 признак) нескольких сайтов, которые Alice 
    использует замето реже, чем остальные люди '''

    sites = df[['site%s' % i for i in range(1, 11)]].values
    result = [0]*sites.shape[0]

    for i, session in enumerate(sites):
        for site in session:
            if site in list_of_sites:
                result[i] += 1
    
    X = sp.hstack([X_sparse, np.array(result).reshape(-1, 1)])
    return X

In [13]:
''' Оказалось, что признак плохой и приводит к переобучению '''

def add_start_hour(df, X_sparse):
    ''' Добавляет час начала сессии (1 признак) '''

    start_hour = df['start_hour']
    X = sp.hstack([X_sparse, start_hour.values.reshape(-1, 1)])
    return X
    

In [14]:
def add_day_of_week(df, X_sparse):
    ''' Добавляет день начала сессии (1 признак) '''

    day_of_week = df['day_of_week']
    X = sp.hstack([X_sparse, day_of_week.values.reshape(-1, 1)])
    return X

Далее закомментированы определения функций, добавляющих плохие признаки.

In [15]:
'''
def add_num_of_unique(df, X_sparse):
    # Добавляет количество уникальных сайтов в сессии (1 признак)

    num_of_unique = df['#unique_sites']
    X = sp.hstack([X_sparse, num_of_unique.values.reshape(-1, 1)])
    return X
'''

"\ndef add_num_of_unique(df, X_sparse):\n    # Добавляет количество уникальных сайтов в сессии (1 признак)\n\n    num_of_unique = df['#unique_sites']\n    X = sp.hstack([X_sparse, num_of_unique.values.reshape(-1, 1)])\n    return X\n"

In [16]:
'''
from sklearn.preprocessing import StandardScaler

def add_session_timespan_scaled(df, X_sparse):
    session_timespan = df['session_timespan_scaled']
    X = sp.hstack([X_sparse, session_timespan.values.reshape(-1, 1)])
    return X
    
scaler_sess_timespan = StandardScaler(with_std=False, with_mean=False)
train_df['session_timespan_scaled'] = scaler_sess_timespan.fit_transform(train_df['session_timespan'].values.reshape(-1, 1))
test_df['session_timespan_scaled'] = scaler_sess_timespan.transform(test_df['session_timespan'].values.reshape(-1, 1))
'''

"\nfrom sklearn.preprocessing import StandardScaler\n\ndef add_session_timespan_scaled(df, X_sparse):\n    session_timespan = df['session_timespan_scaled']\n    X = sp.hstack([X_sparse, session_timespan.values.reshape(-1, 1)])\n    return X\n    \nscaler_sess_timespan = StandardScaler(with_std=False, with_mean=False)\ntrain_df['session_timespan_scaled'] = scaler_sess_timespan.fit_transform(train_df['session_timespan'].values.reshape(-1, 1))\ntest_df['session_timespan_scaled'] = scaler_sess_timespan.transform(test_df['session_timespan'].values.reshape(-1, 1))\n"

### Применение преобразованных наборов данных для классификации

В данном разделе были обучены LogisticRegressionCV на двух тренировочных наборах данных:
- К тренировочным и тестовым наборам, преобразованным при помощи CountVectorizer и TfidfVectorizer добавлены индикаторы утра, дня, вечера и ночи.
- Определены показатели метрики roc-auc.
- Обученные LogisticRegressionCV с найденным коэффициентом регуляризации прменены для классификации сессий в соответствующих тестовых наборах данных.
- Отправлены посылки в соревновании на Kaggle.
- Сделаны выводы.


In [17]:
time_split = TimeSeriesSplit(n_splits=10) # для валидации
y_train = train_df['target'].astype('int').values

In [18]:
X_train_cnt_1f = add_time_features(train_df.fillna(0), X_train)
X_test_cnt_1f = add_time_features(test_df.fillna(0), X_test)

X_train_tfidf_1f = add_time_features(train_df.fillna(0), X_train_tfidf)
X_test_tfidf_1f = add_time_features(test_df.fillna(0), X_test_tfidf)

In [19]:
%%time
logit_c_values = np.linspace(0.1, 5, 10)

logit_grid_searcher_cnt = LogisticRegressionCV(Cs=logit_c_values, 
                                            solver='liblinear', 
                                            random_state=17,
                                            cv=time_split,
                                            n_jobs=-1, scoring='roc_auc',
                                            verbose=3,
                                            max_iter=2000)
logit_grid_searcher_cnt.fit(X_train_cnt_1f, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   38.2s remaining:  1.5min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  1.5min remaining:   39.6s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.5min finished


[LibLinear]Wall time: 2min 39s


LogisticRegressionCV(Cs=array([0.1       , 0.64444444, 1.18888889, 1.73333333, 2.27777778,
       2.82222222, 3.36666667, 3.91111111, 4.45555556, 5.        ]),
                     cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
                     max_iter=2000, n_jobs=-1, random_state=17,
                     scoring='roc_auc', solver='liblinear', verbose=3)

In [20]:
logit_mean_cv_scores_cnt = logit_grid_searcher_cnt.scores_[1].mean(axis=0)
print(logit_mean_cv_scores_cnt.max(), 
      logit_grid_searcher_cnt.Cs_[logit_mean_cv_scores_cnt.argmax()])

0.9166806003300598 0.6444444444444445


In [21]:
logit_test_pred_cnt= logit_grid_searcher_cnt.predict_proba(X_test_cnt_1f)[:, 1]
write_to_submission_file(logit_test_pred_cnt, 'submit_cnt.csv') # 0.93980

In [22]:
%%time
logit_c_values = np.linspace(0.1, 5, 10)

logit_grid_searcher_tfidf = LogisticRegressionCV(Cs=logit_c_values, 
                                            solver='liblinear', 
                                            random_state=17,
                                            cv=time_split,
                                            n_jobs=-1, scoring='roc_auc',
                                            verbose=3,
                                            max_iter=2000)
logit_grid_searcher_tfidf.fit(X_train_tfidf_1f, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   16.6s remaining:   38.8s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   29.2s remaining:   12.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   36.2s finished


[LibLinear]Wall time: 39.6 s


LogisticRegressionCV(Cs=array([0.1       , 0.64444444, 1.18888889, 1.73333333, 2.27777778,
       2.82222222, 3.36666667, 3.91111111, 4.45555556, 5.        ]),
                     cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
                     max_iter=2000, n_jobs=-1, random_state=17,
                     scoring='roc_auc', solver='liblinear', verbose=3)

In [23]:
logit_mean_cv_scores_tfidf = logit_grid_searcher_tfidf.scores_[1].mean(axis=0)
print(logit_mean_cv_scores_tfidf.max(), 
      logit_grid_searcher_tfidf.Cs_[logit_mean_cv_scores_tfidf.argmax()])

0.9245491008379615 4.455555555555556


In [24]:
logit_test_pred_tfidf = logit_grid_searcher_tfidf.predict_proba(X_test_tfidf_1f)[:, 
1]
write_to_submission_file(logit_test_pred_tfidf, 'submit_tfidf.csv') # 0.94368

Поскольку более высокому roc-auc в leaderboard у TfidfVectorizer соответствует более высокое значение данной метрики и на валидации, то можно сделать вывод об удачном выборе метода валидации. В дальнейшем будут использоваться наборы данных, преобразованные при помощи TfidfVectorizer.

In [25]:
X_train_1f = X_train_tfidf_1f
X_test_1f = X_test_tfidf_1f

### Применение новых фичей для классификации

In [26]:
X_train_2f = add_session_timespan(train_df, X_train_1f)
X_test_2f = add_session_timespan(test_df, X_test_1f)

In [27]:
X_train_3f = feature_short_session(train_df, X_train_2f)
X_test_3f = feature_short_session(test_df, X_test_2f) 

In [28]:
X_train_4f = add_top_sites(train_df, X_train_3f)
X_test_4f = add_top_sites(test_df, X_test_3f)

In [29]:
X_train_5f = add_reverse_top_sites(train_df, X_train_4f)
X_test_5f = add_reverse_top_sites(test_df, X_test_4f) 

In [30]:
X_train_6f = feature_middle_session(train_df, X_train_5f)
X_test_6f = feature_middle_session(test_df, X_test_5f)

In [44]:
# Пример кросс-валидации
logit_c_values = np.linspace(3, 5, 50)

logit_final = LogisticRegressionCV(Cs=logit_c_values, 
                                   solver='liblinear', 
                                   random_state=17,
                                   cv=time_split,
                                   n_jobs=-1, scoring='roc_auc',
                                   verbose=3,
                                   penalty='l2')
logit_final.fit(X_train_6f, y_train)

logit_mean_cv_scores = logit_final.scores_[1].mean(axis=0)
print(logit_mean_cv_scores.max(), logit_final.Cs_[logit_mean_cv_scores.argmax()])

# 0.9321137656775317 3.6530612244897958 # 0.95044

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  4.1min remaining:  9.6min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  6.8min remaining:  2.9min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  9.4min finished


[LibLinear]0.9321137656775317 3.6530612244897958


In [45]:
logit_final = LogisticRegression(C=3.6530612244897958, 
                                 solver='liblinear', 
                                 random_state=17, 
                                 n_jobs=-1, 
                                 penalty='l2')
logit_final.fit(X_train_6f, y_train)

LogisticRegression(C=3.6530612244897958, n_jobs=-1, random_state=17,
                   solver='liblinear')

In [46]:
logit_final_cv_score = cross_val_score(logit_final, 
                                       X_train_6f, 
                                       y_train, 
                                       scoring='roc_auc', 
                                       cv=time_split).mean()
print('final cv score:', logit_final_cv_score)

final cv score: 0.9310553238208839


In [47]:
logit_test_pred_final = logit_final.predict_proba(X_test_6f)[:, 1]
write_to_submission_file(logit_test_pred_final, 'subm_logit_final.csv')

### Заключение

Score на публичной части тестовых данных: **0.95044**. Результат далёк от лучших позиций в рейтинге, однако целью проделанной работы являлось знакомство с площадкой Kaggle, получение навыков обработки и визуализации данных, а также изучение основ машинного обучения.

В качестве шагов для улучшения качества построенной модели в дальнейшем будут проделаны следующие шаги:
* применение других методов машинного обучения (бустинг над решающими деревьями, метод ближайших соседей);
* улучшения в схеме кросс-валидации (замечено, что не всегда рост roc-auc на кросс-валидации соответствует росту roc-auc на публичной части тестовых данных);
* проверка на переобученность модели при использовании отдельных признаков.